## Hotspot of Mineral using Hyperspectral data

### Create powerful user interfaces and convert the jupyter notebook/ python scripts into interactive apps and need not worry about views, templates, and then JS to make it interactive


In [5]:
!pip install fastapi uvicorn pydantic numpy streamlit earthengine-api


     |████████████████████████████████| 401 kB 833 kB/s eta 0:00:01
     |████████████████████████████████| 96 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 194 kB 911 kB/s eta 0:00:01
     |████████████████████████████████| 126 kB 236 kB/s eta 0:00:01
     |████████████████████████████████| 11.9 MB 90 kB/s eta 0:00:011    |████▍                           | 1.6 MB 631 kB/s eta 0:00:17     |█████████████████▊              | 6.6 MB 2.2 MB/s eta 0:00:03
     |████████████████████████████████| 181 kB 18 kB/s eta 0:00:011
     |████████████████████████████████| 81 kB 94 kB/s eta 0:00:011
     |████████████████████████████████| 139 kB 98 kB/s eta 0:00:01
     |████████████████████████████████| 85 kB 335 kB/s eta 0:00:01
     |████████████████████████████████| 220 kB 463 kB/s eta 0:00:01
     |████████████████████████████████| 50 kB 647 kB/s eta 0:00:01


### Create the Mineral Calculator Module:

First, make sure the functions to load hyperspectral data and detect mineral hotspots are properly defined. Ensure that the Earth Engine API is initialized only once during application startup.

In [1]:
import numpy as np
import ee

In [2]:
ee.Authenticate()

In [2]:
# Initialize the Earth Engine library
ee.Initialize()

print("Earth Engine initialized.")


EEException: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, or ee.Authenticate() in Python, and then retry.

In [7]:
# hyperspectral_analysis.py

import numpy as np
import ee

# Initialize the Earth Engine library
ee.Initialize()

def load_hyperspectral_data(datacollection, date):
    """
    Load hyperspectral data from Google Earth Engine for a specified time frame.

    Args:
    datacollection (str): The name of the Earth Engine ImageCollection.
    date (str): The date for which to extract the hyperspectral data (format: 'YYYY-MM-DD').

    Returns:
    numpy.ndarray: The hyperspectral data array.
    """
    collection = ee.ImageCollection(datacollection).filterDate(date)
    image = collection.first()
    url = image.getDownloadURL()
    data_collection = np.load(url)
    return data_collection

def detect_mineral_hotspots(data_collection, mineral_signature):
    """
    Detect mineral hotspots based on the hyperspectral data and mineral signature.

    Args:
    data_collection (numpy.ndarray): The hyperspectral data array.
    mineral_signature (float): The signature value of the mineral to detect.

    Returns:
    numpy.ndarray: The binary array indicating the presence of mineral hotspots.
    """
    # Example calculation: You can replace this with actual processing logic.
    hotspots = (data_collection == mineral_signature).astype(int)
    return hotspots


EEException: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, or ee.Authenticate() in Python, and then retry.

### Serve the Function from the Module Using FastAPI
Next, create a FastAPI application that uses these functions. Define the necessary Pydantic models for input validation and set up the API endpoints.

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from hyperspectral_analysis import load_hyperspectral_data, detect_mineral_hotspots

class UserInput(BaseModel):
    datacollection: str
    date: str
    mineral_signature: float

app = FastAPI()

@app.post("/detect-hotspots/")
async def detect_hotspots(user_input: UserInput):
    try:
        data_collection = load_hyperspectral_data(user_input.datacollection, user_input.date)
        hotspots = detect_mineral_hotspots(data_collection, user_input.mineral_signature)
        return {"hotspots": hotspots.tolist()}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)

### Create a Streamlit Application
Finally, create a Streamlit application to provide a user interface that interacts with the FastAPI backend.

In [ ]:
import streamlit as st
import requests

st.title("Mineral Hotspot Detector")

datacollection = st.text_input("Enter the ImageCollection name (e.g., 'COPERNICUS/S2'):")
date = st.text_input("Enter the date (format: YYYY-MM-DD):")
mineral_signature = st.number_input("Enter the mineral signature value:")

if st.button("Detect Hotspots"):
    response = requests.post("http://127.0.0.1:8000/detect-hotspots/", json={
        "datacollection": datacollection,
        "date": date,
        "mineral_signature": mineral_signature
    })
    if response.status_code == 200:
        hotspots = response.json()["hotspots"]
        st.write("Hotspots detected:", hotspots)
    else:
        st.write("Error in detecting hotspots:", response.json()["detail"])

### Run the Applications


In [ ]:
uvicorn main:app --reload


In [1]:
streamlit run app.py